#**Bing Image Creator DALL-E 3 Batch v1.4**
For more details: [GitHub](https://github.com/meap158/bing_image_creator_dalle3_batch)

---
#### **How to Obtain Bing Cookie**
To use Bing Image Creator, you need to obtain the necessary cookie value. Follow these steps:

1. Navigate to Bing Image Creator: https://www.bing.com/images/create.
2. Log in to your Bing account.
3. Open the browser's developer tools:
   - Press `F12` or right-click on the page and select "Inspect" or "Inspect Element."
4. In the Developer Tools, go to the "Application" tab.
5. Under "Cookies," find and select `https://www.bing.com`.
6. Look for the `_U` cookie.
7. Copy the value of the `_U` cookie.
---
#### **How to Use**
1. Input your `_U` cookie value into the `bing_cookie` form.
2. Add prompts to `'prompts.txt'` (open with [Text Editor](https://texteditor.co/)) in the `'bing_image_creator_dalle3_batch'` folder on Google Drive.
3. Press `Ctrl + F9` to run all cells.

In [ ]:
# @title Mount Google Drive, create 'bing_image_creator_dalle3_batch' folder and 'prompts.txt'

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/gdrive')

# Set the path for the new folder
folder_name = 'bing_image_creator_dalle3_batch'
folder_path = '/content/gdrive/My Drive/' + folder_name

# Check if the folder exists, and create it if not
if not os.path.exists(folder_path):
    os.makedirs(folder_path, exist_ok=True)
    print(f"Folder '{folder_name}' created in My Drive.")
else:
    print(f"Folder '{folder_name}' already exists in My Drive.")

# Create an empty file named prompts.txt
prompts_file_path = os.path.join(folder_path, 'prompts.txt')

if not os.path.exists(prompts_file_path):
    with open(prompts_file_path, 'w') as prompts_file:
        prompts_file.write("a glass skull smoking a perfect cuban cigar")
    print(f"'prompts.txt' created in '{folder_name}'.")
else:
    print(f"File 'prompts.txt' already exists in '{folder_name}'.")

In [ ]:
# @title Install Dependencies

!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

!pip install requests
!pip install selenium
!pip install seleniumbase

In [ ]:
# @title Generate

import os
import random
import string
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from seleniumbase import Driver
from urllib.parse import urlparse, urlunparse

from google.colab import userdata
from google.colab import drive


bing_cookie = ""  # @param {type:"string"}
start_at_line = 1  # @param {type:"integer"}


class BingImageCreator:
    def __init__(self, prompts, cookie_value=None, save_folder="images"):
        self.prompts = prompts if isinstance(prompts, list) else [prompts]
        self.cookie_value = cookie_value or bing_cookie or userdata.get("bing_cookie")
        self.save_folder = save_folder
        # Setup driver
        self.driver = Driver(headless2=True, no_sandbox=True)

    def set_prompts(self, new_prompts):
        self.prompts = new_prompts if isinstance(new_prompts, list) else [new_prompts]

    def set_save_folder(self, new_save_folder):
        self.save_folder = new_save_folder

    def navigate_to_bing_create_page(self):
        self.driver.get("https://www.bing.com/images/create")

    def set_cookie(self):
        cookie = {"name": "_U", "value": self.cookie_value}
        self.driver.add_cookie(cookie)

    def enter_prompt(self, prompt):
        # Find the search box by its ID and enter a new prompt
        search_box = self.driver.find_element("id", "sb_form_q")
        search_box.clear()  # Clear existing text
        search_box.send_keys(prompt)
        time.sleep(3)
        search_box.send_keys(Keys.RETURN)  # Press Enter
        time.sleep(1)  # Introduce a small delay (1 second) for stability

    def wait_for_loader(self):
        try:
            # Check if the prompt has been blocked (presence of class "block_icon")
            if self.driver.find_elements(By.CLASS_NAME, "block_icon"):
                # Print the error messages
                error_mt_elements = self.driver.find_elements(By.CLASS_NAME, "gil_err_mt")
                error_sbt_elements = self.driver.find_elements(By.CLASS_NAME, "gil_err_sbt1")
                if error_mt_elements:
                    print("Error Message (gil_err_mt):", error_mt_elements[0].text)
                if error_sbt_elements:
                    print("Error Message (gil_err_sbt1):", error_sbt_elements[0].text)
                return False
            # Find the div with id "giloader"
            self.driver.find_element(By.ID, "giloader")
            # Check if boosts have run out (data-tb attribute of the element with id "reward_c" is 0)
            reward_element = self.driver.find_element(By.ID, "reward_c")
            data_tb_value = reward_element.get_attribute("data-tb")
            # Default wait time
            time_to_wait_seconds = 30  # Adjust the timeout as needed
            if data_tb_value == "0":
                # If data-tb is 0, get wait time from the text of the span with id "gi_rmtime" e.g. "1 min wait", "45 sec wait"
                time.sleep(1)
                # Find all elements with ID "gi_rmtime"
                wait_time_elements = self.driver.find_elements(By.ID, "gi_rmtime")
                if wait_time_elements:
                    # If at least one element is found, get wait time from the text of the first element
                    wait_time_text = wait_time_elements[0].text
                    # Split the wait time text into words
                    wait_time_words = wait_time_text.split()
                    # Extract the numerical value and unit from the wait time text
                    value = int(wait_time_words[0])
                    unit = wait_time_words[1]
                    # Convert minutes or seconds to seconds and add it to the wait time
                    if unit == "min":
                        time_to_wait_seconds += value * 60
                    elif unit == "sec":
                        time_to_wait_seconds += value
                else:
                    # If no elements are found, default to 1 minute wait
                    time_to_wait_seconds += 60
            # Wait until the style becomes "display: none;"
            wait = WebDriverWait(self.driver, time_to_wait_seconds)
            print("time_to_wait_seconds:", time_to_wait_seconds)
            wait.until(EC.invisibility_of_element_located((By.ID, "giloader")))
            time.sleep(3)
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    @staticmethod
    def remove_middle_text(text, max_length, indicator="___"):
        """
        Remove excess characters from the middle of the text if its length exceeds the specified maximum.
        """
        if len(text) > max_length:
            excess_chars = len(text) - max_length
            chars_to_remove_from_middle = (
                excess_chars + 1
            ) // 2  # Calculate the number of characters to remove from the middle
            start_index = (len(text) - max_length - chars_to_remove_from_middle) // 2
            end_index = start_index + max_length
            text = text[:start_index] + indicator + text[end_index:]
        return text

    @classmethod
    def sanitize_filename(self, filename):
        """
        Sanitize the given filename to make it compatible across different operating systems.
        """
        valid_chars = set("-_.() %s%s" % (string.ascii_letters, string.digits))
        sanitized_filename = "".join(c if c in valid_chars else "_" for c in filename)
        # Limit the filename length to 196 characters
        max_length = 196
        sanitized_filename = self.remove_middle_text(sanitized_filename, max_length)
        # Remove leading and trailing underscores
        sanitized_filename = sanitized_filename.strip("_")
        # Truncate the filename from the right to a maximum length of 196 characters
        sanitized_filename = (
            sanitized_filename[-max_length:]
            if len(sanitized_filename) > max_length
            else sanitized_filename
        )
        # If the filename is empty, generate a random string and append to "unnamed"
        if not sanitized_filename:
            random_string = "".join(
                random.choice(string.ascii_letters + string.digits) for _ in range(8)
            )
            sanitized_filename = f"unnamed_{random_string}"
        return sanitized_filename

    def download_images(self):
        try:
            # Select .mimg elements that are descendants of .img_cont
            image_elements = self.driver.find_elements(
                By.CSS_SELECTOR, ".img_cont .mimg"
            )
            image_urls = [element.get_attribute("src") for element in image_elements]
            # Create the save folder if it doesn't exist
            os.makedirs(self.save_folder, exist_ok=True)
            # Download images using alt values as sanitized file names
            for url, element in zip(image_urls, image_elements):
                alt = element.get_attribute("alt")
                sanitized_alt = self.sanitize_filename(alt)
                # Remove unwanted parameters from the URL
                parsed_url = urlparse(url)
                new_url = urlunparse(
                    (
                        parsed_url.scheme,
                        parsed_url.netloc,
                        parsed_url.path,
                        parsed_url.params,
                        "",
                        "",
                    )
                )
                new_url += "?pid=ImgGn"
                # Retrieve the full image content
                response = requests.get(new_url)
                # Modify the file path to save the images to Google Drive
                file_path = os.path.join(self.save_folder, f"{sanitized_alt}.jpg")
                # Write the image content to the file
                with open(file_path, "wb") as f:
                    f.write(response.content)
                print(f'Image saved to "{file_path}".')
        except Exception as e:
            print(f"Error: {e}")

    def run(self):
        for prompt in self.prompts:
            self.navigate_to_bing_create_page()
            self.set_cookie()
            self.driver.refresh()
            self.enter_prompt(prompt)
            if self.wait_for_loader():
                self.download_images()
        self.driver.quit()


# Change directory to the BingImageCreator folder
google_driver_folder = "/content/gdrive/My Drive/bing_image_creator_dalle3_batch"
os.chdir(google_driver_folder)
# Get a list of all text files in the folder
txt_files = [file for file in os.listdir() if file.endswith(".txt")]
# Loop through each text file
for txt_file in txt_files:
    # Remove the ".txt" extension to get the subfolder name
    subfolder_name = os.path.splitext(txt_file)[0]
    # Create the subfolder path
    subfolder_path = os.path.join(google_driver_folder, subfolder_name)
    # Read the lines from the text file
    txt_file_path = os.path.join(google_driver_folder, txt_file)
    with open(txt_file_path, "r") as file:
        lines = file.read().splitlines()
        if not isinstance(start_at_line, int) or start_at_line < 1:
            start_at_line = 1
        for line_index, line in enumerate(lines, start_at_line):
            if line.strip():
                image_creator = BingImageCreator(line, save_folder=subfolder_path)
                print(f"Prompt {line_index}:", line)
                print("Folder path:", subfolder_path)
                image_creator.run()
                print()
